In [2]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,AIMessage
import os   
import dotenv

In [ ]:
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_classic.agents import initialize_agent,AgentType
llm = ChatGroq(model = "llama-3.3-70b-versatile",verbose=True,max_tokens=500,api_key=GROQ_API_KEY)

In [12]:
from typing_extensions import TypedDict
class State(TypedDict):
    topic : str
    initial_setting : str
    character_actions : str
    ending : str

In [13]:
def initial_setting(state:State):
    msg = llm.invoke(f'write an initial premise of a story with two characters about the topic: {state["topic"]}')
    return {"initial_setting":msg.content}
def character_actions(state:State):
    msg = llm.invoke(f'write the middle part of the story in which some actions are taken by the charactters based on this given premise : {state["initial_setting"]}')
    return {'character_actions':msg.content}
def ending(state:State):
    msg = llm.invoke(f'write a happy ending for this story : {state["character_actions"]}')
    return {"ending":msg.content}

In [15]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)
builder.add_node("initial_setting",initial_setting)
builder.add_node("character_actions",character_actions)
builder.add_node("ending",ending)

builder.add_edge(START,"initial_setting")
builder.add_edge("initial_setting","character_actions")
builder.add_edge("character_actions",'ending')
builder.add_edge("ending",END)

graph = builder.compile()

In [22]:
for event in graph.stream({"topic": "space exploration"}):
    if "initial_setting" in event:
        print("\n--- INITIAL SETTING ---\n")
        print(event["initial_setting"]["initial_setting"])
    elif "character_actions" in event:
        print("\n--- CHARACTER ACTIONS ---\n")
        print(event["character_actions"]["character_actions"])
    elif "ending" in event:
        print("\n--- ENDING ---\n")
        print(event["ending"]['ending'])


--- INITIAL SETTING ---

As the stars whizzed by like diamonds on velvet, Captain Jaxon "Jax" Lee gazed out at the endless expanse of space from the cockpit of their ship, the Aurora. Beside him, his trusted pilot and friend, Dr. Sophia Patel, worked tirelessly to navigate the uncharted territories of the distant planet, Kepler-62f. Their mission, dubbed "New Horizon," aimed to explore the mysterious planet's potential for human habitation, but as they delved deeper into the unknown, they began to unravel a sinister secret that threatened to upend the entire space program. With their ship's resources dwindling and the planet's hostile environment closing in, Jax and Sophia must rely on each other's expertise and trust to survive, but as the silence of space seemed to grow thicker, they couldn't shake the feeling that they were not alone in the universe.

--- CHARACTER ACTIONS ---

As the ship's life support systems began to falter, Jax and Sophia knew they had to act quickly to ensure